In [1]:
import sys, os, git, time

import MDAnalysis as mda
from MDAnalysis.analysis import distances

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from MinimizeEnergy import MinimizeEnergy

import numpy as np

In [11]:
ti = time.time()

self = MinimizeEnergy(number_atoms=[1000], minimization_steps=10)
self.perform_energy_minimization()

tf = time.time()

print("time =", np.round(tf-ti,2), "s")

step epot maxF
0 441498455692.347 138509004429810.938
time = 90.65 s


In [14]:
ti = time.time()

r_ij = mda.analysis.distances.self_distance_array(self.atoms_positions,
                                                    self.box_size)
energy_potential = np.sum(4*self.array_epsilon_ij*(np.power(self.array_sigma_ij/r_ij, 12)-np.power(self.array_sigma_ij/r_ij, 6)))

tf = time.time()
print(tf-ti, energy_potential)

0.08799910545349121 9072004.845813803


In [30]:
mda.analysis.distances.contact_matrix_pbc(self.atoms_positions, "Numpy", self.box_size, 12)

ValueError: Buffer dtype mismatch, expected 'float' but got 'double'

In [20]:
ti = time.time()

energy_potential = 0
for Ni, position_i, sigma_i, epsilon_i, neighbor_of_i in zip(np.arange(self.total_number_atoms-1),
                                            self.atoms_positions,
                                            self.atoms_sigma,
                                            self.atoms_epsilon,
                                            self.neighbor_lists):

    # Read information about neighbors
    positions_j = self.atoms_positions[neighbor_of_i]
    sigma_j = self.atoms_sigma[neighbor_of_i]
    epsilon_j = self.atoms_epsilon[neighbor_of_i]
    # Measure distances and other cross parameters
    rij_xyz = (np.remainder(position_i - positions_j
                            + self.box_size[:3]/2., self.box_size[:3])
                            - self.box_size[:3]/2.)
    rij = np.linalg.norm(rij_xyz, axis=1)
    sigma_ij = (sigma_i+sigma_j)/2
    epsilon_ij = (epsilon_i+epsilon_j)/2
    # Measure potential
    energy_potential += np.sum(4*epsilon_ij*((sigma_ij/rij)**12-(sigma_ij/rij)**6))

tf = time.time()
print(tf-ti, energy_potential)

0.27906298637390137 9072004.019642465


In [18]:
4*epsilon_ij*((sigma_ij/rij)**12-(sigma_ij/rij)**6)

array([-1.84772812e-04, -4.66298076e-05, -2.21226500e-04, -1.46320357e-04,
       -2.75214871e-04, -5.92206434e-04, -2.61560580e-04, -3.32219850e-03,
       -2.49263682e-02, -5.23509219e-04, -1.90454294e-04, -6.17785015e-04,
       -3.42884622e-05, -5.97547721e-04, -6.16947552e-05, -5.93345424e-04,
       -2.89379299e-04, -6.22104390e-04, -1.03311494e-04, -1.79309580e-04,
       -1.53774307e-04, -2.62728020e-05, -7.83956636e-03, -4.60764855e-05,
       -8.78950319e-05, -2.32848717e-02, -3.37399529e-04, -6.13028887e-05,
        7.80642725e+00, -4.36718246e-04, -4.34641818e-05, -1.16702080e-03,
       -1.07020764e-02, -3.08810908e-04, -2.71881989e-05, -6.85893879e-05,
       -2.70318571e-03, -4.01765637e-04, -1.78270204e-03, -3.02516139e-04,
       -5.19140439e-05, -2.06795735e-04, -3.81865706e-04, -1.47779851e-04,
       -6.16468970e-05, -3.45047820e-04, -1.91199165e-04, -8.74663781e-05,
       -1.66556893e-03, -1.81594419e-04, -2.04867033e-04, -9.52717652e-05,
       -5.02323787e-04, -